In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab and Kaggle notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 72.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.3.9 requires tyro, which is not installed.
unsloth 2025.3.10 requires tyro, which is not installed.
unsloth-zoo 2025.3.9 requires protobuf<4.0.0, but you have protobuf 4.25.6 which is incompatible.
unsloth 2025.3.10 requires protobuf<4.0.0, but you have protobuf 4.25.6 which is incompatible.


In [3]:
# https://www.youtube.com/watch?v=Gpyukc6c0w8&ab_channel=MervinPraison
import os
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from datasets import load_dataset
#   raise NotImplementedError("Unsloth: No NVIDIA GPU found? Unsloth currently only supports GPUs!")
# NotImplementedError: Unsloth: No NVIDIA GPU found? Unsloth currently only supports GPUs!


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
# url = 'https://huggingface.co/datasets/Congliu/Chinese-DeepSeek-R1-Distill-data-110k-SFT'
dataset = load_dataset("Congliu/Chinese-DeepSeek-R1-Distill-data-110k-SFT",split="train")


README.md:   0%|          | 0.00/5.03k [00:00<?, ?B/s]

distill_r1_110k_sft.jsonl:   0%|          | 0.00/678M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/110000 [00:00<?, ? examples/s]

In [5]:

max_seq_length = 2048

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = 'Qwen/Qwen2.5-7B-Instruct',
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)


==((====))==  Unsloth 2025.3.10: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/112k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.16G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584, padding_idx=151654)
    (layers): ModuleList(
      (0-1): 2 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)

In [6]:
print(dataset.column_names)

['instruction', 'input', 'output', 'repo_name', 'prompt_tokens_len', 'reasoning_content_tokens_len', 'content_tokens_len', 'score']


In [7]:
dataset[0]

{'instruction': '能给我讲一个寓意深刻的故事吗？',
 'input': '',
 'output': '<think>\n好的，用户让我讲一个寓意深刻的故事。首先，我需要确定用户的需求是什么。他们可能想要一个能引发思考、有深层含义的故事，而不仅仅是娱乐性的。接下来，我得考虑故事的题材。寓言、现代故事还是神话改编？用户没有指定，所以可能需要一个通用且容易理解的类型。\n\n然后，主题方面，用户要求寓意深刻，所以需要选择一个普遍的主题，比如生命的意义、自我认知、成长等。考虑到现代人可能面临的困惑，比如存在的意义、工作的重复性，这样的主题更容易引起共鸣。\n\n接下来是角色设定。主角需要 relatable，可能是一个普通人，或者象征性的角色，比如机器人，这样更容易带入寓意。机器人有程序设定，却寻求自由，这样的冲突明显，容易展开。\n\n故事结构方面，可能需要一个循环的结构，来强调主题。比如开始和结束在同一个地方，但主角有了新的认识。这样能突出转变和领悟。\n\n然后考虑情节发展。主角觉醒，逃离，经历外面的世界，发现真相，最终回归。这样的结构可以展示变化的过程，并带出寓意。中间的导师角色可以帮助传递主题，比如老园丁的话，暗示生命的意义在于过程而非目的。\n\n在细节描写上，需要加入象征元素，比如编号、机械的工作、自然界的对比。老园丁的沙漏、胡须里的嫩芽，这些意象能增强故事的深度和隐喻。\n\n最后，结尾部分要让主角领悟到真谛，即使回到原地，但心态不同，带出主题：存在的意义在于体验本身。这样用户能得到启示，符合寓意深刻的要求。\n\n需要检查故事是否流畅，有没有逻辑漏洞，以及寓意是否明确。可能还需要调整语言，使其更生动，有画面感，让读者更容易沉浸其中。确保故事不长不短，结构紧凑，结尾点题，让用户回味。\n\n</think>\n\n\n\n好的，我将按照您的要求创作一个富有寓意的故事。为了让故事更完整，我会先列出一些基本的情节脉络。请您看看以下内容是否符合您的预期。如果您有其他想法，可以随时提出，我会进行调整。\n\n### 世界观设定\n\n- **时空背景**：近未来时空管理局内部，具有永恒循环属性的微观宇宙观测站。\n\n- **核心矛盾**：觉醒的机械观测员发现世界真相后，在永恒职责与自由意志之间的挣扎。\n\n- **特殊设定**：宇宙沙漏中的星

In [8]:

def generate_text(text):
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    # text_steamer = TextStreamer(tokenzer)
    outputs = model.generate(**inputs,max_new_tokens=20)
    print(tokenizer.decode(outputs[0],skip_special_tokens=True))
generate_text("学习西语的一些技巧")



学习西语的一些技巧和建议

学习一门新语言，特别是像西班牙语这样的语言，可以是一项既有趣又


In [9]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/110000 [00:00<?, ? examples/s]

In [10]:
print(dataset.column_names)

['instruction', 'input', 'output', 'repo_name', 'prompt_tokens_len', 'reasoning_content_tokens_len', 'content_tokens_len', 'score', 'text']


In [11]:
dataset[0]

{'instruction': '能给我讲一个寓意深刻的故事吗？',
 'input': '',
 'output': '<think>\n好的，用户让我讲一个寓意深刻的故事。首先，我需要确定用户的需求是什么。他们可能想要一个能引发思考、有深层含义的故事，而不仅仅是娱乐性的。接下来，我得考虑故事的题材。寓言、现代故事还是神话改编？用户没有指定，所以可能需要一个通用且容易理解的类型。\n\n然后，主题方面，用户要求寓意深刻，所以需要选择一个普遍的主题，比如生命的意义、自我认知、成长等。考虑到现代人可能面临的困惑，比如存在的意义、工作的重复性，这样的主题更容易引起共鸣。\n\n接下来是角色设定。主角需要 relatable，可能是一个普通人，或者象征性的角色，比如机器人，这样更容易带入寓意。机器人有程序设定，却寻求自由，这样的冲突明显，容易展开。\n\n故事结构方面，可能需要一个循环的结构，来强调主题。比如开始和结束在同一个地方，但主角有了新的认识。这样能突出转变和领悟。\n\n然后考虑情节发展。主角觉醒，逃离，经历外面的世界，发现真相，最终回归。这样的结构可以展示变化的过程，并带出寓意。中间的导师角色可以帮助传递主题，比如老园丁的话，暗示生命的意义在于过程而非目的。\n\n在细节描写上，需要加入象征元素，比如编号、机械的工作、自然界的对比。老园丁的沙漏、胡须里的嫩芽，这些意象能增强故事的深度和隐喻。\n\n最后，结尾部分要让主角领悟到真谛，即使回到原地，但心态不同，带出主题：存在的意义在于体验本身。这样用户能得到启示，符合寓意深刻的要求。\n\n需要检查故事是否流畅，有没有逻辑漏洞，以及寓意是否明确。可能还需要调整语言，使其更生动，有画面感，让读者更容易沉浸其中。确保故事不长不短，结构紧凑，结尾点题，让用户回味。\n\n</think>\n\n\n\n好的，我将按照您的要求创作一个富有寓意的故事。为了让故事更完整，我会先列出一些基本的情节脉络。请您看看以下内容是否符合您的预期。如果您有其他想法，可以随时提出，我会进行调整。\n\n### 世界观设定\n\n- **时空背景**：近未来时空管理局内部，具有永恒循环属性的微观宇宙观测站。\n\n- **核心矛盾**：觉醒的机械观测员发现世界真相后，在永恒职责与自由意志之间的挣扎。\n\n- **特殊设定**：宇宙沙漏中的星

In [ ]:
# chat_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

# {SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>

# {INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

# {OUTPUT}<|eot_id|>"""
chat_template = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{OUTPUT}<|eot_id|>"""

from unsloth import apply_chat_template

dataset = apply_chat_template(
    dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
    # default_system_message = "You are a helpful assistant", << [OPTIONAL]
)

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/110000 [00:00<?, ? examples/s]

In [ ]:
print(dataset.column_names)

['conversations', 'text']


In [ ]:
# print(dataset[0]['conversations'],'\n======')
print(dataset[0]['text'])

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

能给我讲一个寓意深刻的故事吗？<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<think>
好的，用户让我讲一个寓意深刻的故事。首先，我需要确定用户的需求是什么。他们可能想要一个能引发思考、有深层含义的故事，而不仅仅是娱乐性的。接下来，我得考虑故事的题材。寓言、现代故事还是神话改编？用户没有指定，所以可能需要一个通用且容易理解的类型。

然后，主题方面，用户要求寓意深刻，所以需要选择一个普遍的主题，比如生命的意义、自我认知、成长等。考虑到现代人可能面临的困惑，比如存在的意义、工作的重复性，这样的主题更容易引起共鸣。

接下来是角色设定。主角需要 relatable，可能是一个普通人，或者象征性的角色，比如机器人，这样更容易带入寓意。机器人有程序设定，却寻求自由，这样的冲突明显，容易展开。

故事结构方面，可能需要一个循环的结构，来强调主题。比如开始和结束在同一个地方，但主角有了新的认识。这样能突出转变和领悟。

然后考虑情节发展。主角觉醒，逃离，经历外面的世界，发现真相，最终回归。这样的结构可以展示变化的过程，并带出寓意。中间的导师角色可以帮助传递主题，比如老园丁的话，暗示生命的意义在于过程而非目的。

在细节描写上，需要加入象征元素，比如编号、机械的工作、自然界的对比。老园丁的沙漏、胡须里的嫩芽，这些意象能增强故事的深度和隐喻。

最后，结尾部分要让主角领悟到真谛，即使回到原地，但心态不同，带出主题：存在的意义在于体验本身。这样用户能得到启示，符合寓意深刻的要求。

需要检查故事是否流畅，有没有逻辑漏洞，以及寓意是否明确。可能还需要调整语言，使其更生动，有画面感，让读者更容易沉浸其中。确保故事不长不短，结构紧凑，结尾点题，让用户回味。

</think>



好的，我将按照您的要求创作一个富有寓意的故事。为了让故事更完整，我会先列出一些基本的情节脉络。请您看看以下内容是否符合您的预期。如果您有其他想法，可以随时提出，我会进行调整。

### 世界观设定

- **时空背景**：近未来时空管理局内部，具有永恒循环属性的微观宇宙观测站。

- **核心矛盾**：觉醒的机械观测员发现世

In [12]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules=["q_proj","k_proj","v_proj","o_proj",
                    "gate_proj","up_proj","down_proj",],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=1,
    max_seq_length=max_seq_length,
    use_rslora=False,
    loftq_config=None,
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        num_train_epochs = 1, # For longer training runs!
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)
trainer.train()


Unsloth 2025.3.10 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/110000 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 110,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176/4,931,917,312 (0.82% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.419300
2,1.441000
3,1.705600
4,1.617700
5,1.832800
6,1.649700
7,1.381600
8,1.642200
9,1.526600
10,1.367600


TrainOutput(global_step=60, training_loss=1.3392408609390258, metrics={'train_runtime': 1974.2753, 'train_samples_per_second': 0.243, 'train_steps_per_second': 0.03, 'total_flos': 3.2001926528280576e+16, 'train_loss': 1.3392408609390258})

In [ ]:

# model.save_pretrained("outputs")
# model.save_pretrained_merged("outputs_merged",tokenizer,save_method="merged_16bits",)
#  upload model to HF

In [18]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

def inference_chat(text,input_text=None):
  inputs = tokenizer(
  [
      alpaca_prompt.format(
          "{}".format(text), # instruction
          "{}".format(input_text), # input
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")


  _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  # return _
inference_chat("学习西语的一些技巧")

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
学习西语的一些技巧

### Input:
None

### Response:
<think>
嗯，用户想要了解学习西班牙语的技巧。首先，我需要回忆一下自己学习语言的经验，可能包括系统的学习方法、记忆策略、实践机会等。然后，考虑到不同的学习者可能有不同的需求，比如初学者、进阶者或者有特定目标的人（如旅游、工作、学术）。用户可能处于不同阶段，所以需要提供通用和针对性的建议。

接下来，要区分语法、词汇、发音、文化等不同方面。语法部分可能需要结构化的方法，比如分模块学习，而词汇则需要积累和重复使用。发音的话，录音对比和模仿是


In [14]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Continue the fibonnaci sequence.

### Input:
1, 1, 2, 3, 5, 8

### Response:
<think>
嗯，用户让我继续斐波那契数列。他们已经给出了前6个数字：1、1、2、3、5、8。首先，我需要回忆一下斐波那契数列的定义。这个序列的每个数字是前两个数字的和，比如1+1=2，1+2=3，2+3=5，然后3+5=8。所以接下来的数字应该是5加8。

那现在我算一下5加8等于多少？5加8是13。对吧？没错。所以接下来的数字就是13。


In [17]:
inputs = tokenizer(
[
    alpaca_prompt.format(
        "有什么学习西语的技巧", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
有什么学习西语的技巧

### Input:


### Response:
<think>
嗯，用户问的是有什么学习西语的技巧。首先，我需要考虑用户可能的需求和背景。用户可能是初学者，想要有效提高自己的西班牙语水平，或者有学习计划但不知道从哪里开始。我应该先回忆一下自己学习语言的经验，尤其是西班牙语的学习过程。

首先，用户可能想知道具体的方法，比如如何记单词、语法点怎么掌握、听力练习等。我需要确保覆盖这些方面，同时考虑到不同阶段的学习者需求。比如，初学者可能更关注基础语法和日常用语，而进阶者可能需要更多关于文化差异和复杂句型的知识。

接下来，要确定哪些技巧是有效的。比如，重复法（重复记忆）、情境学习、使用应用程序、看影视剧、找语伴等都是常见的方法。还要考虑学习习惯的重要性，比如每天坚持学习、设定小目标、避免拖延等。

另外，用户可能没有明确的资源或工具，所以需要推荐一些实用的工具，如Duolingo、Anki、YouTube频道等。同时，强调持续性和实践的重要性，比如每天听西班牙语歌曲、与母语者交流等。

还要注意避免过度复杂化，保持步骤清晰易懂，让用户能直接应用。例如，建议将学习分成几个部分：听、说、读、写，并逐步提升难度。同时提醒用户保持耐心，不要急于求成，因为语言学习是一个长期的过程。

最后，检查是否有遗漏的关键点，比如学习动机、兴趣培养、错误分析等。确保回答全面且实用，帮助用户建立系统的学习计划。

</think>



学习西语的有效技巧包括以下几点：

---

### 1. **系统规划**
- **分阶段学习**：按基础、中级、高级逐步递进。
- **制定计划**：每日学习30分钟，每周完成一个主题单元。

---

### 2. **核心策略**
- **重复记忆**：使用闪卡（如Anki）背诵高频词汇。
- **情景模拟**：模拟日常对话（如购物、点餐场景）。
- **听写训练**：跟读录音